In [1]:
import numpy as np

In [2]:
hard_biosent = np.load("../resources/artificial_inds_hard_key_v5.npy",allow_pickle=True)
hard_tfidf = np.load("../resources/artificial_inds_easy_key_v5.npy",allow_pickle=True)

In [3]:
hard_biosent[0]

array([['Breast Cancer Screening', '01_01.txt', '0.534118115901947', ...,
        '0.49915446313505907', '2.613237842091487', '0'],
       ['Ovarian Cancer Screening', '04_11.txt', '0.5126306414604187',
        ..., '0.5068788481271703', '2.5722638823140977', '51'],
       ['Breast Imaging of Pregnant and Lactating Women', '01_02.txt',
        '0.497243732213974', ..., '0.4634212727459952',
        '2.4033595249392645', '1'],
       ...,
       ['Imaging of Mesenteric Ischemia', '11_08.txt',
        '0.323648065328598', ..., '0.17980315496514998',
        '1.0215844723579515', '171'],
       ['Pelvic Floor Dysfunction', '04_12.txt', '0.3007028102874756',
        ..., '0.17241342259443493', '1.013586175233027', '52'],
       ['Penetrating Trauma–Lower Abdomen and Pelvis', '10_09.txt',
        '0.2872193455696106', ..., '0.17338413348191395',
        '0.9780466064928', '152']], dtype='<U120')

In [4]:
hard_tfidf[0]

array([['Breast Imaging of Pregnant and Lactating Women', '01_02.txt',
        '0.4927057921886444', ..., '0.4954610896774535',
        '2.4755672909821285', '1'],
       ['Breast Cancer Screening', '01_01.txt', '0.4871990978717804',
        ..., '0.4618334362227252', '2.4101364878769314', '0'],
       ['Stage I Breast Cancer: Initial Workup and Surveillance for Local Recurrence and Distant Metastases in Asymptomatic Women',
        '01_09.txt', '0.4944331645965576', ..., '0.4352562255450799',
        '2.2411942859854443', '8'],
       ...,
       ['Tinnitus', '07_24.txt', '0.3057592213153839', ...,
        '0.17592038406092247', '1.0035441128360725', '111'],
       ['Radiologic Management of Lower-Extremity Venous Insufficiency',
        '11_15.txt', '0.2616093158721924', ..., '0.10929150274278421',
        '0.7885968652630359', '178'],
       ['Radiologic Management of Lower-Extremity Venous Insufficiency',
        '12_12.txt', '0.2616093158721924', ..., '0.10929150274278421',
      

In [5]:
import csv

in_f = '../resources/artificial_indications_filled_v2.csv'
reader = csv.reader(open(in_f))

hard_queries = []
hard_etiologies = []
categories = []
for row in reader:
    etiologies = []
    if row[2] != 'narr_names': # Don't include header row
#         print(row)
        [_,name,narr_name, simple, difficult, _,_,_,_,_,_,_,_,_,_,_] = row
        
        etiologies = row[5:14]
        print(etiologies)
        hard_etiologies.append(etiologies)
        
        category = row[14]
        print(category)
        categories.append(category)
        if difficult != '':
            hard_queries.append([name, narr_name, difficult, '', '', ''])

FileNotFoundError: [Errno 2] No such file or directory: '../resources/artificial_indications_filled_v2.csv'

In [ ]:
in_f = '/mnt/c/users/gunu1/documents/ucsf_rad/acr_project/resources/relevance_criteria.csv'
reader = csv.reader(open(in_f))

criteria = []
for row in reader:
    if row[0] != 'file_code': # Don't include header row
        criteria.append(row)
        

In [ ]:
import math

def calc_relevance(doc_index,query_index):
    rel = 0
    
    doc_eti = criteria[doc_index][2:-2]
    query_eti = hard_etiologies[query_index]
    
    for i in range(len(doc_eti)):
        if query_eti[i] == "x" and doc_eti[i] == "x" and i!=3:
            rel = 1
    
    
    if doc_eti[3] == "x" and categories[query_index] == criteria[doc_index][-2]:
        rel += 1
    
    return rel/2
            

names = list(np.load('../resources/names.npy'))

tfidf_rel = []
biosent_rel = []

for max_counter in range(1,51):
    for query_i,query in enumerate(hard_queries):
        this_tfidf_rel = 0
        this_biosent_rel = 0

        #iterating in one query
        counter = 0
        for rank_i,eachdoc in enumerate(hard_tfidf[query_i]):
    #         print(rank_i)
    #         print(hard_tfidf[query_i][rank_i][0])

            if(hard_tfidf[query_i][rank_i][0] != hard_tfidf[query_i][rank_i+1][0]):
                this_rel = calc_relevance(names.index(hard_tfidf[query_i][rank_i][1]),query_i)
                this_tfidf_rel += (this_rel)/(math.log(counter+2,2))
    #             print(this_tfidf_rel)
                counter += 1

            if counter >= max_counter:
                break;


        counter = 0
        for rank_i,eachdoc in enumerate(hard_biosent[query_i]):
    #         print(rank_i)
    #         print(hard_biosent[query_i][rank_i][0])

            if(hard_biosent[query_i][rank_i][0] != hard_biosent[query_i][rank_i+1][0]):
                this_rel = calc_relevance(names.index(hard_biosent[query_i][rank_i][1]),query_i)
                this_biosent_rel += (this_rel)/(math.log(counter+2,2))
                counter += 1

            if counter >= max_counter: #max = 188
                break;

        tfidf_rel.append(this_tfidf_rel)
        biosent_rel.append(this_biosent_rel)

    print(np.mean(biosent_rel))

In [ ]:
ct = 0
for x in range(len(tfidf_rel)):
    if tfidf_rel[x] > biosent_rel[x]:
        print(x, tfidf_rel[x]-biosent_rel[x])
        ct += 1
        
ct